# **Doublet-Source Panel Method**

In [21]:
include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")
using Revise
using .PanelMethods: Uniform2D, doublet_source_panels, collocation_point, split_panels, grid_data, aero_coefficients!, pressure_coefficient, ×
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST, naca4, linspace
using BenchmarkTools

In [22]:
alpha_u = [0.2, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.2, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
airfoil = kulfan_CST(alphas, dzs, 0.0, 100);

In [23]:
uniform = Uniform2D(1.0, 5.0)
panels = doublet_source_panels(airfoil);

In [24]:
@time cl = aero_coefficients!(panels, uniform)
println("Lift Coefficient: $cl")

  0.016145 seconds (742 allocations: 2.223 MiB)
Lift Coefficient: -0.6125083409773331


In [ ]:
using Plots

In [6]:
plotly();

In [9]:
x_domain, y_domain = (-1, 2), (-1, 1)
grid_size = 50
x_dom, y_dom = linspace(x_domain..., grid_size), linspace(y_domain..., grid_size)
grid = x_dom × y_dom

vels, pots = grid_data(panels, grid)

([(0.005780771850842317, 0.00398137367248205) (0.006015962554743768, 0.003693313027173848) … (0.002571218938199807, 0.004660122598373516) (0.0025111029103879515, 0.003774163558466966); (0.005811446226883961, 0.004232441091771532) (0.006054945161190017, 0.003946669778762895) … (0.0024340549648832364, 0.004912028167282738) (0.0023762622286091707, 0.003932416459334264); … ; (-0.004328916418371859, 0.009321634217647354) (-0.00470551762888897, 0.009434890596920917) … (-0.0024298522077993985, -0.00901786523675297) (-0.001593696951637927, -0.009091566557125303); (-0.004584155070249178, 0.009002636669222307) (-0.004970975651115279, 0.009091667308851014) … (-0.0028303086534154165, -0.008798785523601133) (-0.0020212897648342163, -0.008870413209245628)], [0.0016581608110076294 0.001758112845071513 … 0.0006584003333216238 0.0006262130456365593; 0.001657815771973944 0.0017620435881005839 … 0.0006413925665330191 0.0006095078100938515; … ; -0.0015741957646489319 -0.0016783367349317794 … -0.0007080899

In [11]:
cp = pressure_coefficient.(uniform.mag, vels);

In [12]:
plot(airfoil[:,1], airfoil[:,2], markershape = :circle, aspect_ratio = :equal)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
upper_panels, lower_panels = split_panels(panels)
plot([ (first ∘ collocation_point)(panel) for panel in upper_panels ], [ panel.cp for panel in upper_panels ], 
     label = "Upper", markershape = :circle)
plot!([ (first ∘ collocation_point)(panel) for panel in lower_panels ], [ panel.cp for panel in lower_panels ], 
      label = "Lower", markershape = :circle, yaxis = (:flip))

<!DOCTYPE html>
 
 
 Plots.jl

In [14]:
p1 = contour(x_dom, y_dom, cp, fill = true)
plot(p1)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), color = "black", aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl